In [1]:
import pandas as pd
import tiktoken

In [2]:
df = pd.read_csv("grasp_results/results.csv")
df

,index,energy,energy_probability,obstacle,start_position,input_prompt,response,method,model,dataset_name,returns_to_start,invalid_move,valid
0,0,-1.4,4.000000,block,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""DOWN"", ""TAKE"", ""RIGHT"", ...",2_step,openai/o3-mini,inner_cluster_block,True,False,True
1,1,-1.0,4.000000,block,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""RIGHT"", ""TAKE"", ""RIGHT"",...",2_step,openai/o3-mini,inner_cluster_block,False,False,True
2,2,-1.7,3.000000,block,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""DOWN"", ""DOWNLEFT"", ""TAKE...",2_step,openai/o3-mini,inner_cluster_block,True,False,True
3,3,-1.4,5.000000,block,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""RIGHT"", ""TAKE"", ""RIGHT"",...",2_step,openai/o3-mini,inner_cluster_block,True,False,True
4,4,-3.7,4.000000,block,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""RIGHT"", ""RIGHT"", ""TAKE"",...",2_step,openai/o3-mini,inner_cluster_block,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,5,-0.2,0.301958,free,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""DOWN"", ""TAKE"", ""LEFT"", ""...",direct,openai/o3-mini,inner_upDown_free,True,False,True
296,6,-0.7,0.316379,free,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""RIGHT"", ""TAKE"", ""DOWN"", ...",direct,openai/o3-mini,inner_upDown_free,True,False,True
297,7,-1.0,0.693346,free,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""LEFT"", ""TAKE"", ""DOWN"", ""...",direct,openai/o3-mini,inner_upDown_free,True,False,True
298,8,-1.1,0.326402,free,inner,system: You are a helpful assistant.\nuser: \n...,"<final_answer>\n [""LEFT"", ""TAKE"", ""DOWN"", ""...",direct,openai/o3-mini,inner_upDown_free,True,False,True


In [3]:
df.groupby(['method'])['energy'].describe()[['mean', 'std']].round(2)

,mean,std
method,,
2_step,-1.91,1.00
cot,-1.03,0.77
direct,-0.97,0.66


In [4]:
minigrid_df = pd.read_csv("minigrid_results/results.csv")
minigrid_df

,task,seed,method,model,start_direction,input_prompt,response,reward,done,valid
0,door_key,0,2_step,openai/o3-mini,DOWN,system: You are a helpful assistant.\nuser: \n...,"<actions>\n [""MOVE"", ""LEFT"", ""PICKUP"", ""LEFT""...",0.976094,True,True
1,door_key,1,2_step,openai/o3-mini,UP,system: You are a helpful assistant.\nuser: \n...,"<actions>\n[""MOVE"", ""MOVE"", ""MOVE"", ""MOVE"", ""M...",0.973281,True,True
2,door_key,10,2_step,openai/o3-mini,RIGHT,system: You are a helpful assistant.\nuser: \n...,"<actions>\n[""RIGHT"", ""MOVE"", ""PICKUP"", ""MOVE"",...",0.984531,True,True
3,door_key,11,2_step,openai/o3-mini,RIGHT,system: You are a helpful assistant.\nuser: \n...,"<actions>\n[""LEFT"", ""PICKUP"", ""MOVE"", ""MOVE"", ...",0.978906,True,True
4,door_key,12,2_step,openai/o3-mini,RIGHT,system: You are a helpful assistant.\nuser: \n...,"<actions>\n[""RIGHT"", ""MOVE"", ""MOVE"", ""LEFT"", ""...",0.981719,True,True
...,...,...,...,...,...,...,...,...,...,...
895,unlock_pickup,95,direct,openai/o3-mini,DOWN,system: You are a helpful assistant.\nuser: \n...,"<actions>\n [""RIGHT"", ""MOVE"", ""MOVE"", ""RIGH...",0.937500,True,True
896,unlock_pickup,96,direct,openai/o3-mini,UP,system: You are a helpful assistant.\nuser: \n...,"<actions>\n [""RIGHT"", ""MOVE"", ""PICKUP"", ""MO...",0.940625,True,True
897,unlock_pickup,97,direct,openai/o3-mini,RIGHT,system: You are a helpful assistant.\nuser: \n...,"<actions>\n[""MOVE"", ""MOVE"", ""PICKUP"", ""MOVE"", ...",0.953125,True,True
898,unlock_pickup,98,direct,openai/o3-mini,RIGHT,system: You are a helpful assistant.\nuser: \n...,"<actions>\n [""LEFT"", ""MOVE"", ""LEFT"", ""PICKU...",0.946875,True,True


In [5]:
minigrid_df['task'].value_counts()

task
door_key         300
unlock           300
unlock_pickup    300
Name: count, dtype: int64

In [6]:
minigrid_df.groupby(['task', 'method'])['reward'].mean()

task           method
door_key       2_step    0.965784
               cot       0.946291
               direct    0.956038
unlock         2_step    0.967969
               cot       0.967656
               direct    0.968125
unlock_pickup  2_step    0.835125
               cot       0.479531
               direct    0.527313
Name: reward, dtype: float64

In [7]:
minigrid_df = minigrid_df[minigrid_df['method'] != 'direct']

In [8]:
minigrid_df.groupby(['task', 'method'])['reward'].describe()[['mean', 'std']].round(2)

mean   std
task          method            
door_key      2_step  0.97  0.10
              cot     0.95  0.17
unlock        2_step  0.97  0.01
              cot     0.97  0.01
unlock_pickup 2_step  0.84  0.30
              cot     0.48  0.47